<a href="https://colab.research.google.com/github/HaJunYoo/Inflearn_RecSys/blob/main/3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 모든 사용자 간 평가의 유사도 계산

2. ⇒ 추천 대상과 다른 사용자간 유사도 추출

3. ⇒ 추천 대상이 평가하지 않은 아이템에 대한 예상 평가값 계산
   (평가값 = 다른 사용자 평가 X 다른 사용자 유사도)

4. ⇒ 아이템 중에서 예상 평가값 가장 높은 N개 추천

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 단순 협업 필터링

In [ ]:
# 데이터 불러오기
import os
import pandas as pd

base_src = '/content/drive/MyDrive/추천시스템/RecoSys/Data'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv( u_user_src, 
                    sep = '|' ,
                    names = u_cols,
                    encoding = 'latin-1'
                    )

# users = users.set_index('user_id')

# u.item을 DataFrame
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date',
'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
'FilmNoir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
# movies = movies.set_index('movie_id')

# u_data
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
                      sep = '\t',
                      names = r_cols,
                      encoding = 'latin-1')
# ratings = ratings.set_index('user_id')

In [ ]:
# 정확도를 계산하는 함수
import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
def score(model):
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  # model에 user과 movie를 반복적으로 넣으면서 예측 결과 array를 구성
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)


In [ ]:
# 데이터 셋 만들기
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.25,
                                                    # 층화 추출  
                                                    # 라벨 비율 반영해서 추출 
                                                    stratify = y)
# movie_id, user_id full matrix 구성
ratings_matrix = x_train.pivot(
    index = 'user_id',
    columns = 'movie_id',
    values = 'rating'
)

In [ ]:
#### 코사인 유사도 계산 ###
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)

user_similarity = pd.DataFrame(user_similarity, 
                               index = ratings_matrix.index,
                               columns = ratings_matrix.index)

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.141118,0.048865,0.076208,0.289239,0.311489,0.316675,0.233042,0.040972,0.312593,...,0.300429,0.111322,0.206016,0.142628,0.155229,0.090572,0.185342,0.106170,0.113083,0.343651
2,0.141118,1.000000,0.058214,0.106766,0.043130,0.208908,0.060576,0.087162,0.083894,0.057704,...,0.051334,0.288381,0.283981,0.356054,0.256175,0.202573,0.172502,0.107576,0.081385,0.061430
3,0.048865,0.058214,1.000000,0.263699,0.027021,0.035571,0.058181,0.072101,0.000000,0.054609,...,0.017582,0.053228,0.159348,0.053951,0.117288,0.032980,0.127023,0.103623,0.085388,0.033472
4,0.076208,0.106766,0.263699,1.000000,0.013724,0.040147,0.085517,0.224852,0.060804,0.083645,...,0.070691,0.050063,0.097249,0.188470,0.106588,0.041358,0.131693,0.207915,0.169481,0.064761
5,0.289239,0.043130,0.027021,0.013724,1.000000,0.229199,0.265624,0.173352,0.007788,0.172550,...,0.308174,0.080796,0.058698,0.077250,0.100412,0.055093,0.223770,0.109854,0.129398,0.277192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.090572,0.202573,0.032980,0.041358,0.055093,0.094044,0.110878,0.066955,0.000000,0.034412,...,0.064627,0.264358,0.187899,0.213550,0.372679,1.000000,0.075286,0.162519,0.041561,0.122803
940,0.185342,0.172502,0.127023,0.131693,0.223770,0.258277,0.224098,0.133139,0.068114,0.262611,...,0.228539,0.078514,0.149980,0.146166,0.127106,0.075286,1.000000,0.161946,0.190787,0.210839
941,0.106170,0.107576,0.103623,0.207915,0.109854,0.060099,0.050840,0.158951,0.136060,0.048054,...,0.048735,0.174867,0.245751,0.193065,0.315261,0.162519,0.161946,1.000000,0.029019,0.047127


In [ ]:
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.141118,0.048865,0.076208,0.289239,0.311489,0.316675,0.233042,0.040972,0.312593,...,0.300429,0.111322,0.206016,0.142628,0.155229,0.090572,0.185342,0.106170,0.113083,0.343651
2,0.141118,1.000000,0.058214,0.106766,0.043130,0.208908,0.060576,0.087162,0.083894,0.057704,...,0.051334,0.288381,0.283981,0.356054,0.256175,0.202573,0.172502,0.107576,0.081385,0.061430
3,0.048865,0.058214,1.000000,0.263699,0.027021,0.035571,0.058181,0.072101,0.000000,0.054609,...,0.017582,0.053228,0.159348,0.053951,0.117288,0.032980,0.127023,0.103623,0.085388,0.033472
4,0.076208,0.106766,0.263699,1.000000,0.013724,0.040147,0.085517,0.224852,0.060804,0.083645,...,0.070691,0.050063,0.097249,0.188470,0.106588,0.041358,0.131693,0.207915,0.169481,0.064761
5,0.289239,0.043130,0.027021,0.013724,1.000000,0.229199,0.265624,0.173352,0.007788,0.172550,...,0.308174,0.080796,0.058698,0.077250,0.100412,0.055093,0.223770,0.109854,0.129398,0.277192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.090572,0.202573,0.032980,0.041358,0.055093,0.094044,0.110878,0.066955,0.000000,0.034412,...,0.064627,0.264358,0.187899,0.213550,0.372679,1.000000,0.075286,0.162519,0.041561,0.122803
940,0.185342,0.172502,0.127023,0.131693,0.223770,0.258277,0.224098,0.133139,0.068114,0.262611,...,0.228539,0.078514,0.149980,0.146166,0.127106,0.075286,1.000000,0.161946,0.190787,0.210839
941,0.106170,0.107576,0.103623,0.207915,0.109854,0.060099,0.050840,0.158951,0.136060,0.048054,...,0.048735,0.174867,0.245751,0.193065,0.315261,0.162519,0.161946,1.000000,0.029019,0.047127


In [ ]:
### 주어진 영화의 (movie_id) 가중 평균 rating을 계산하는 함수 ####
def CF_simple(user_id, movie_id):
  if movie_id in ratings_matrix.columns:
    # 어떤 한 유저와 다른 유저들 간의 유사도
    sim_scores = user_similarity[user_id].copy()
    # 영화 평점 
    movie_ratings = ratings_matrix[movie_id].copy()
    # 해당 영화를 평가하지 않은 사람들 인덱스 담아서 유사도 점수에서 제거
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    # 해당 영화를 평가하지 않은 사람들 index에 해당되는 유사도 점수 제거
    sim_scores = sim_scores.drop(none_rating_idx)
    # 유저 간 유사도 점수 * 영화에 대한 유저들의 평점 / 유사도 점수 합
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  
  else:
    mean_rating = 3.0
  return mean_rating

# 평가 경향이 비슷한 유사도가 높은 사용자에게 가중치를 더 주어서 평균을 계산

In [ ]:
### 정확도 계산 ###
score(CF_simple)

1.0201578818792187

In [ ]:
sim_scores = user_similarity[943].copy()
sim_scores

user_id
1      0.343651
2      0.061430
3      0.033472
4      0.064761
5      0.277192
         ...   
939    0.122803
940    0.210839
941    0.047127
942    0.160440
943    1.000000
Name: 943, Length: 943, dtype: float64

In [ ]:
sim_scores.sum()

153.72031286449203

### 이웃을 고려한 CF

In [ ]:
# 정확도를 계산하는 함수
import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
# 유사집단 크기를 위해 neighbor_size 인자값 추가
def score(model, neighbor_size=0):
  # 테스트 데이터의 user id와 movie id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듦
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  # model에 user과 movie를 반복적으로 넣으면서 knn예측 결과 array를 구성
  y_pred = np.array([model(user, movie, neighbor_size) for (user,movie) in id_pairs])
  # 실제 평점값
  y_true = np.array(x_test['rating'])
  # RMSE 계산
  return RMSE(y_true,y_pred)


In [ ]:
# 데이터 셋 만들기
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.25,
                                                    # 층화 추출  
                                                    # 라벨 비율 반영해서 추출 
                                                    stratify = y)
# movie_id, user_id full matrix 구성
ratings_matrix = x_train.pivot(
    index = 'user_id',
    columns = 'movie_id',
    values = 'rating'
)

In [ ]:
#### 코사인 유사도 계산 ###
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 구하기 위해 rating값을 복사하고, 계산 시 NaN 값 에러 대비를 위해 결측치를 0으로 대체 
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 유저 간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)

# 유저 간 유사도를 담은 DF
user_similarity = pd.DataFrame(user_similarity, 
                               index = ratings_matrix.index,
                               columns = ratings_matrix.index)

In [ ]:
## KNN 방식의 Neighbor size를 정해서 예측치를 계산하는 함수 ###
def CF_knn(user_id, movie_id, neighbor_size=0):
  if movie_id in ratings_matrix.columns:
    # 어떤 한 유저와 다른 유저들 간의 유사도
    sim_scores = user_similarity[user_id].copy()
    # 영화 평점 
    movie_ratings = ratings_matrix[movie_id].copy()
    # 해당 영화를 평가하지 않은 사람들 인덱스 담아서 유사도 점수에서 제거
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    # 해당 영화를 평가하지 않은 사람들 index에 해당되는 유사도 점수 제거
    sim_scores = sim_scores.drop(none_rating_idx)

    # 기본 CF
    if neighbor_size == 0:
      # 유저 간 유사도 점수 * 영화에 대한 유저들의 평점 / 유사도 점수 합
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

    else:
      # 영화를 평가한 것이 자기 자신 밖에 없지 않다면 
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        # sim score 오름차순으로 정렬한 인덱스
        user_idx = np.argsort(sim_scores)
        # sim score 높은 순으로 neighbor size 만큼 역순으로 (뒤에가 높음)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

      else:
        mean_rating = 3.0
    
  else:
    mean_rating = 3.0
    
  return mean_rating

# 정확도 계싼
score(CF_knn, neighbor_size=30)

1.0111330207733884

In [ ]:
sim_scores = user_similarity[20].copy()
sim_scores = np.array(sim_scores)
user_idx = np.argsort(sim_scores)
neighbor_size = min(5, len(sim_scores))
print(user_idx)
print(neighbor_size, len(sim_scores))

In [ ]:
sim_scores

In [ ]:
sim_scores = sim_scores[user_idx]
sim_scores

In [ ]:
sim_scores[-neighbor_size:]

array([0.32250569, 0.32427194, 0.32839388, 0.35671904, 1.        ])

In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
### 실제 주어진 사용자에 대해 추천을 받는 기능 구현 ###

# movie_id, user_id full matrix 구성
ratings_matrix = ratings.pivot(
    index = 'user_id',
    columns = 'movie_id',
    values = 'rating'
)

#### 코사인 유사도 계산 ###
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 구하기 위해 rating값을 복사하고, 계산 시 NaN 값 에러 대비를 위해 결측치를 0으로 대체 
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 유저 간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)

# 유저 간 유사도를 담은 DF
user_similarity = pd.DataFrame(user_similarity, 
                               index = ratings_matrix.index,
                               columns = ratings_matrix.index)

In [ ]:
def recom_movie(user_id, n_items, neighbor_size=30):
  user_movie = ratings_matrix.loc[user_id].copy()

  for movie in ratings_matrix.columns:
    # 영화를 이미 본 경우 not null 일 것
    if pd.notnull(user_movie.loc[movie]):
      # 본 것들 제외
      user_movie.loc[movie] = 0
    
    else:
      # 해당 영화에 대해 knn 예측치
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)
  
  # 높은 순, 추천 목록 개수로 가져오기
  movie_sort = user_movie.sort_values(ascending=False)[:n_items]

  # 영화, 영화 이름 series로 뽑아내기
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']

  return recommendations

recom_movie(user_id=729, n_items=5, neighbor_size=30)

movie_id
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
1467                                     Cure, The (1995)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
22                                     Taxi Driver (1976)
Name: title, dtype: object

In [ ]:
## KNN 방식의 Neighbor size를 정해서 예측치를 계산하는 함수 ###
def CF_knn(user_id, movie_id, neighbor_size=0):
  if movie_id in ratings_matrix.columns:
    # 어떤 한 유저와 다른 유저들 간의 유사도
    sim_scores = user_similarity[user_id].copy()
    # 영화 평점 
    movie_ratings = ratings_matrix[movie_id].copy()
    # 해당 영화를 평가하지 않은 사람들 인덱스 담아서 유사도 점수에서 제거
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    # 해당 영화를 평가하지 않은 사람들 index에 해당되는 유사도 점수 제거
    sim_scores = sim_scores.drop(none_rating_idx)

    # 기본 CF
    if neighbor_size == 0:
      # 유저 간 유사도 점수 * 영화에 대한 유저들의 평점 / 유사도 점수 합
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

    else:
      # 영화를 평가한 것이 자기 자신 밖에 없지 않다면 
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        # sim score 오름차순으로 정렬한 인덱스
        user_idx = np.argsort(sim_scores)
        # sim score 높은 순으로 neighbor size 만큼 역순으로 (뒤에가 높음)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

      else:
        mean_rating = 3.0
    
  else:
    mean_rating = 3.0
    
  return mean_rating

# 정확도 계산
# score(CF_knn, neighbor_size=30) 

# neighbor size가 10 ~ 60(10단위)인 경우에 대해서 RMSE를 계산하고 이를 출력한다
for neighbor_size in [10, 20, 30, 40, 50, 60]:
  print(f'Neighbor size = {neighbor_size}: RMSE = {score(CF_knn, neighbor_size=neighbor_size):.4f}')

Neighbor size = 10: RMSE = 0.8076
Neighbor size = 20: RMSE = 0.8767
Neighbor size = 30: RMSE = 0.9042
Neighbor size = 40: RMSE = 0.9191
Neighbor size = 50: RMSE = 0.9284
Neighbor size = 60: RMSE = 0.9349


### 사용자의 평가경향을 고려한 CF

In [ ]:
# 데이터 불러오기
import os
import pandas as pd

base_src = '/content/drive/MyDrive/추천시스템/RecoSys/Data'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv( u_user_src, 
                    sep = '|' ,
                    names = u_cols,
                    encoding = 'latin-1'
                    )

# users = users.set_index('user_id')

# u.item을 DataFrame
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date',
'IMDB URL','unknown','Action','Adventure','Animation',
'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
'FilmNoir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,
                     sep = '|',
                     names = i_cols,
                     encoding='latin-1')
# movies = movies.set_index('movie_id')

# u_data
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
                      sep = '\t',
                      names = r_cols,
                      encoding = 'latin-1')
# ratings = ratings.set_index('user_id')

In [ ]:
# 정확도를 계산하는 함수
import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
# 유사집단 크기를 위해 neighbor_size 인자값 추가
def score(model, neighbor_size=0):
  # 테스트 데이터의 user id와 movie id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듦
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  # model에 user과 movie를 반복적으로 넣으면서 knn예측 결과 array를 구성
  y_pred = np.array([model(user, movie, neighbor_size) for (user,movie) in id_pairs])
  # 실제 평점값
  y_true = np.array(x_test['rating'])
  # RMSE 계산
  return RMSE(y_true,y_pred)

In [ ]:
# 데이터 셋 만들기
from sklearn.model_selection import train_test_split

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.25,
                                                    # 층화 추출  
                                                    # 라벨 비율 반영해서 추출 
                                                    stratify = y)
# movie_id, user_id full matrix 구성
ratings_matrix = x_train.pivot(
    index = 'user_id',
    columns = 'movie_id',
    values = 'rating'
)

In [ ]:
#### 코사인 유사도 계산 ###
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 구하기 위해 rating값을 복사하고, 계산 시 NaN 값 에러 대비를 위해 결측치를 0으로 대체 
matrix_dummy = ratings_matrix.copy().fillna(0)
# 모든 유저 간 코사인 유사도 구함
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)

# 유저 간 유사도를 담은 DF
user_similarity = pd.DataFrame(user_similarity, 
                               index = ratings_matrix.index,
                               columns = ratings_matrix.index)

In [ ]:
ratings_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1670,1672,1674,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,NaN,3.0,3.0,NaN,NaN,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
### 사용자 평가 경향을 고려한 함수 ###
# 사용자별 평균
rating_mean = ratings_matrix.mean(axis=1)
# 사용자별 평점 평균에 대한 편차
rating_bias = (ratings_matrix.T - rating_mean).T

# 사용자 평가 경향을 고려한 함수
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
  if movie_id in ratings_matrix.columns:
    # 어떤 한 유저와 다른 유저들 간의 유사도
    sim_scores = user_similarity[user_id].copy()
    # 영화 평점 
    movie_ratings = rating_bias[movie_id].copy()
    # 해당 영화를 평가하지 않은 사람들 인덱스 담아서 유사도 점수에서 제거
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    # 해당 영화를 평가하지 않은 사람들 index에 해당되는 유사도 점수 제거
    sim_scores = sim_scores.drop(none_rating_idx)

    # 기본 CF
    if neighbor_size == 0:
      # 유저 간 유사도 점수 * 영화에 대한 유저들의 평점 / 유사도 점수 합
      # 예측치는 평점 편차 + 유저별 평점 평균을 더해주어야 한다
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]

    else:
      # 영화를 평가한 것이 자기 자신 밖에 없지 않다면 
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        # sim score 오름차순으로 정렬한 인덱스
        user_idx = np.argsort(sim_scores)
        # sim score 높은 순으로 neighbor size 만큼 역순으로 (뒤에가 높음)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        prediction = rating_mean[user_id]
    
  else:
    prediction = rating_mean[user_id]
    
  return prediction



In [ ]:
# 정확도 계산
score(CF_knn_bias, 30)

3.6218818852547177

In [ ]:
rating_mean

user_id
1      3.568627
2      3.782609
3      2.829268
4      4.166667
5      2.870229
         ...   
939    4.216216
940    3.425000
941    4.062500
942    4.254237
943    3.396825
Length: 943, dtype: float64